In [1]:
!pip uninstall -y torch torchvision torchaudio ultralytics
!pip cache purge


Files removed: 0


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196

In [3]:
!pip install ultralytics==8.1.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.6 MB/s eta 0:00:00


In [4]:
import torch, ultralytics
print("Torch version:", torch.__version__)
print("Ultralytics version:", ultralytics.__version__)


Torch version: 2.5.1+cu121
Ultralytics version: 8.1.0


In [5]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
print("✅ YOLOv8 model loaded successfully!")


100%|██████████| 6.23M/6.23M [00:00<00:00, 339MB/s]

✅ YOLOv8 model loaded successfully!



/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location="cpu"), file  # load

In [7]:
# ============================================================
# 🚀 YOLOv8 Object Detection & Tracking (Error-Free, Colab-Ready)
# ============================================================

!pip uninstall -y ultralytics torch torchvision torchaudio -q
!pip install ultralytics==8.0.196 torch==2.5.1 torchvision torchaudio filterpy==1.4.5 opencv-python-headless==4.10.0.84 --quiet

import torch, cv2, numpy as np, time, os
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter
from google.colab import files

print("✅ Environment ready and compatible!")

# 1️⃣ Upload your video file
uploaded = files.upload()
video_path = next(iter(uploaded))
print(f"🎥 Uploaded: {video_path}")

# 2️⃣ Patch torch.load globally (safe, no recursion)
_old_load = torch.load
def patched_load(*args, **kwargs):
    kwargs["weights_only"] = False
    return _old_load(*args, **kwargs)
torch.load = patched_load

# 3️⃣ Load YOLOv8 model safely
try:
    model = YOLO("yolov8n.pt")
    print("✅ YOLOv8 model loaded successfully!")
except Exception as e:
    raise RuntimeError(f"❌ Failed to load YOLO model: {e}")

# 4️⃣ Define simple Kalman Tracker (for smoother boxes)
class KalmanBoxTracker:
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([
            [1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1],
        ])
        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0,0],
        ])
        self.kf.R *= 10
        self.kf.P *= 1000
        self.kf.Q *= 0.01
        self.kf.x[:4] = np.expand_dims(bbox, axis=1)
        self.id = int(time.time() * 1000) % 1000000

    def update(self, bbox): self.kf.update(bbox)
    def predict(self): self.kf.predict(); return self.kf.x[:4].flatten()

# 5️⃣ Open and process video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("❌ Unable to open video source")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = "output_tracked.mp4"
out = cv2.VideoWriter(out_path, fourcc, 30, (int(cap.get(3)), int(cap.get(4))))

print("🎬 Processing video... please wait")

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    for r in results[0].boxes:
        x1, y1, x2, y2 = r.xyxy[0].cpu().numpy()
        label = model.names[int(r.cls)]
        conf = float(r.conf)
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
print(f"✅ Video processed successfully — saved as {out_path}")

files.download(out_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00

Saving Cars Bollard Wall Crash – BeamNG.drive.mp4 to Cars Bollard Wall Crash – BeamNG.drive.mp4
🎥 Uploaded: Cars Bollard Wall Crash – BeamNG.drive.mp4
✅ YOLOv8 model loaded successfully!
🎬 Processing video... please wait
✅ Video processed successfully — saved as output_tracked.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>